# Test Function for JackOfAllTradesPSO
---
Description:

- Optimization (min)
- Single-objective
- Constraints (no)
---

The general problem statement is given by:

- $f(\mathbf{x}) = \sum_i |x_i|$, with $x_1 \in \Re$, $x_2 \in [-3, +3]$ and $x_3 \in \{a, b, c\}$,
where D is the corresponding dimension. 

assuming a=0, b=1, c=2, then the global minimum found at: $f(0.0, 0, a) = 0$.

### First we import python libraries and set up the directory of our code.

In [19]:
import os, sys
import numpy as np
import cProfile, pstats

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [20]:
# Import main classes.
from star_pso.auxiliary.swarm import Swarm
from star_pso.auxiliary.utilities import BlockType
from star_pso.auxiliary.data_block import DataBlock
from star_pso.auxiliary.jat_particle import JatParticle
from star_pso.engines.jack_of_all_trades_pso import JackOfAllTradesPSO

### Define the test function.

In [21]:
# Auxiliary mapping.
key_to_value = {"a": 0, "b": 1, "c": 2, "d": 5}

# Test function.
def fun_test1(x: np.typing.ArrayLike):
    # Unpack input values.
    x1, x2, x3 = x
    
    # Convert symbols to numerical values.
    x3 = key_to_value[x3]
    
    # Compute the function value.
    f_val = np.sum(np.abs((x1, x2, x3)))

    # Condition for termination.
    solution_found = f_val == 0.0
    
    # Return the solution tuple.
    return -f_val, solution_found
# _end_def_

Here we set the PSO parameters.

In [22]:
# Random number generator.
rng = np.random.default_rng()

# Define the variable set for the categorical
# optimization variable.
var_set = ["a", "b", "c", "d"]

# Size of the variable set.
L = len(var_set)

# Define the number of optimizing variables.
D = 3

# Define the number of particles.
N = min(5*D, 100)

# Initial population.
swarm_t0 = Swarm([JatParticle([DataBlock(rng.uniform(-10.0, +10.0),
                                         BlockType.FLOAT,
                                         lower_bound=-10.0,
                                         upper_bound=+10.0),
                               DataBlock(rng.integers(-3, +3, endpoint=True),
                                         BlockType.INTEGER,
                                         lower_bound=-3,
                                         upper_bound=+3),
                               DataBlock(np.ones(L)/L,
                                         BlockType.CATEGORICAL,
                                         valid_set=var_set)]) for _ in range(N)])

# Create the JackOfAllTradesPSO object that will carry on the optimization.
test_PSO = JackOfAllTradesPSO(initial_swarm = swarm_t0, obj_func = fun_test1)

In [23]:
PROFILE = False

if PROFILE:
    cProfile.run("test_PSO.run(max_it = 500, options = {'w': 0.5, 'c1': 0.1, 'c2': 0.1, 'global_avg': False}, verbose = True)",
                 'output.prof')
else:
    test_PSO.run(max_it = 500, options = {"w": 0.5, "c1": 2.05, "c2": 2.05, "global_avg": True},
                 reset_swarm=True, verbose = True)

Initial f_optimal = -0.4740
Iteration:     1 -> f_optimal = -0.3797
Iteration:    51 -> f_optimal = -0.0017
Iteration:   101 -> f_optimal = -0.0000
Iteration:   151 -> f_optimal = -0.0000
Iteration:   201 -> f_optimal = -0.0000
Iteration:   251 -> f_optimal = -0.0000
Iteration:   301 -> f_optimal = -0.0000
Iteration:   351 -> f_optimal = -0.0000
Iteration:   401 -> f_optimal = -0.0000
Iteration:   451 -> f_optimal = -0.0000
Final f_optimal = -0.0000
run: elapsed time = 1.082 seconds.


### Optimization process.

Here we call the PSO object. We set a number of parameters, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergence, etc.

In [24]:
# Extract the optimal solution from the PSO.
optimal_solution = test_PSO.swarm.best_particle()

# Display the (final) optimal value.
print(f"Optimum Found: {optimal_solution.value}\n")

# Display each gene value separately.
for i, xi in enumerate(optimal_solution.position, start=1):
    print(f"x{i} = {xi}")
# _end_for_

print("\nBest sampled position:")
print(test_PSO.stats["x_best"][-1])

Optimum Found: -7.049959210179883e-19

x1 = -7.049959210179883e-19
x2 = 0
x3 = [0.72566943 0.05653506 0.16896629 0.04882922]

Best sampled position:
[-7.049959210179883e-19, 0, np.str_('a')]


### End of file

In [25]:
# p = pstats.Stats('output.prof')

In [26]:
# p.sort_stats('cumtime').print_stats(20)